In [ ]:
!wget -O dataset.zip https://s3-ap-southeast-1.amazonaws.com/he-public-data/datasetb2d9982.zip

In [ ]:
!unzip dataset.zip

### Importing Required Libraries

In [ ]:
!pip3 install datasets torch numpy pandas bs4 lxml nltk scikit-learn

In [1]:
import datasets
import nltk
import numpy as np
import os
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.linear_model import LinearRegression

_ = nltk.download('all')

/Users/software/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package averaged_perce

[nltk_data]    |   Package nombank.1.0 is already up-to-date!
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package nonbreaking_prefixes is already up-to-date!
[nltk_data]    | Downloading package nps_chat to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package nps_chat is already up-to-date!
[nltk_data]    | Downloading package omw to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package omw is already up-to-date!
[nltk_data]    | Downloading package omw-1.4 to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package omw-1.4 is already up-to-date!
[nltk_data]    | Downloading package opinion_lexicon to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package opinion_lexicon is already up-to-date!
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    | 

[nltk_data]    |   Package wordnet is already up-to-date!
[nltk_data]    | Downloading package wordnet2021 to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package wordnet2021 is already up-to-date!
[nltk_data]    | Downloading package wordnet2022 to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package wordnet2022 is already up-to-date!
[nltk_data]    | Downloading package wordnet31 to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package wordnet31 is already up-to-date!
[nltk_data]    | Downloading package wordnet_ic to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package wordnet_ic is already up-to-date!
[nltk_data]    | Downloading package words to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package words is already up-to-date!
[nltk_data]    | Downloading package ycoe to
[nltk_data]    |     /Users/software/nltk_data...
[nltk_data]    |   Package ycoe is already up

In [2]:
def group_by_product_id(df):
    """
    Group a dataframe by PRODUCT_TYPE_ID and return a dictionary with PRODUCT_TYPE_ID as key and an
    array of the dataframe with that product_id as value.
    
    Parameters:
        df (pandas.DataFrame): the dataframe to group
        
    Returns:
        dict: a dictionary with product_id as key and an array of the dataframe with that product_id as value
    """
    result = {}
    for product_id in df['PRODUCT_TYPE_ID'].unique():
        result[product_id] = df[df['PRODUCT_TYPE_ID'] == product_id]
    return result

def clean_data(df):
    # remove HTML tags from the text
    # print('Removing HTML tags')
    # df['TEXT'] = df['TEXT'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

    # remove any non-alphanumeric characters from the text
    print('Removing non-alphanumeric characters')
    df['TEXT'] = df['TEXT'].str.replace(r'[^a-zA-Z0-9\*\'\"\s]', '')

    # convert all text to lowercase
    print('Lowercasing dataset')
    df['TEXT'] = df['TEXT'].str.lower()

    # tokenize the text
    print('Tokenizing dataset based on spaces')
    df['TEXT'] = df['TEXT'].str.split()

    # remove stop words using NLTK
    print('Removing stop words')
    stop_words = set(stopwords.words('english'))
    df['TEXT'] = df['TEXT'].apply(lambda x: [word for word in x if word not in stop_words])

    # join the tokens back into a single string
    print('Joining back tokens into single string')
    df['TEXT'] = df['TEXT'].str.join(' ')

    return df

def preprocess(df):
    df = df.fillna("")
    df["BULLET_POINTS"] = df["BULLET_POINTS"].apply(lambda x: " ".join(str(x).strip()[1 : -1].split(",")))
    df["TEXT"] = df["TITLE"] + " " + df["BULLET_POINTS"] + " " + df["DESCRIPTION"]
    df = clean_data(df)
    return df

In [3]:
DATASET_FOLDER = "./dataset"
train_file = os.path.join(DATASET_FOLDER, "train.csv")
test_file = os.path.join(DATASET_FOLDER, "test.csv")

In [4]:
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

In [5]:
train_df = preprocess(train_df)
test_df = preprocess(test_df)

Removing non-alphanumeric characters


/var/folders/ck/gs22xnt513x04n1m13hg3w6h0000gr/T/ipykernel_33606/1209565415.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['TEXT'] = df['TEXT'].str.replace(r'[^a-zA-Z0-9\*\'\"\s]', '')


Lowercasing dataset
Tokenizing dataset based on spaces
Removing stop words
Joining back tokens into single string
Removing non-alphanumeric characters


/var/folders/ck/gs22xnt513x04n1m13hg3w6h0000gr/T/ipykernel_33606/1209565415.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['TEXT'] = df['TEXT'].str.replace(r'[^a-zA-Z0-9\*\'\"\s]', '')


Lowercasing dataset
Tokenizing dataset based on spaces
Removing stop words
Joining back tokens into single string


In [7]:
chunk_size = 100000

def batch_process(filename):
    dataset_filename = os.path.join(DATASET_FOLDER, filename)
    header = pd.read_csv(dataset_filename, nrows = 0)

    with open(f'cleaned_{filename}', 'w', newline = '') as file:
        header.to_csv(file, index = False)
        index = 0

        for chunk in pd.read_csv(dataset_filename, chunksize = chunk_size):
            print(f'Processing {filename} Chunk {index}')
            index += 1
            preprocess(chunk).to_csv(file, header = False, index = False)

In [ ]:
batch_process("train.csv")
batch_process("test.csv")

In [ ]:
!zip -r -9 cleaned *.csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp cleaned.zip /content/drive/MyDrive

In [6]:
train_df

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,TEXT
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,LUXURIOUS & APPEALING: Beautiful custom-made c...,,1650,2125.980000,artzfolio tulip flowers blackout curtain door ...
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,Harry Potter Hedwig Pyjamas (6-16 Yrs) 100% co...,,2755,393.700000,marks spencer girls' pyjama sets t862561cnavy ...
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,Loud Dual Tone Trumpet Horn Compatible With S...,"Specifications: Color: Red, Material: Aluminiu...",7537,748.031495,priknik horn red electric air horn compressor ...
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,Made By 95%cotton and 5% Lycra which gives you...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574,alishah women's cotton ankle length leggings c...
4,283658,The United Empire Loyalists: A Chronicle of th...,,,6112,598.424000,united empire loyalists chronicle great migration
...,...,...,...,...,...,...,...
2249693,2422167,Nike Women's As W Ny Df Swsh Hn Kh Bra (CZ7610...,aterial : Polyeste,,3009,1181.100000,nike women's w ny df swsh hn kh bra cz7610010b...
2249694,2766635,"(3PCS) Goose Game Cute Cartoon Enamel Pins, Fu...",❤ [Inspiration] Inspired by the Untitled Goose...,<p><b>[Brand]: </b>XVIEONR</p> <p><br></p> <p>...,3413,125.984252,3pcs goose game cute cartoon enamel pins funny...
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,Dial size: 12 inches in diameter Big clear re...,Wall Clocks Are Very Attractive In Looks And E...,1574,1200.000000,kangroo sweep movement printed wooden wall clo...
2249696,1165754,Electro Voice EKX-BRKT15 | Wall Mount Bracket ...,,,592,2900.000000,electro voice ekxbrkt15 wall mount bracket ekx...


In [11]:
train_df.to_csv("cleaned_train.csv")
test_df.to_csv("cleaned_test.csv")

In [9]:
import spacy
activated = spacy.require_gpu()
nlp = spacy.load("en_core_web_sm")

print(activated)

True



def get_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            #return(f"ent.text -> {ent.text}, ent.start-> {ent.start}, ent.end -> {ent.end}, ent.start_char-> {ent.start_char}, ent.end_char ->{ent.end_char}, ent.label_ ->{ent.label_}")
            return(f"ent.text -> {ent.text}, ent.label_ ->{ent.label_}")
    else:
        return("No entities found")


title_list = train_df["TITLE"][:40000].to_list()
text_list = train_df["TEXT"][:40000].to_list()

ents = []

for element in text_list :
    
    doc = nlp(element)
    ents.append(str(get_ents(doc)))
    
required_entity_dataframe = pd.DataFrame(dict(title_list,ents))


In [ ]:
a = set(test_df["PRODUCT_TYPE_ID"].unique())
b = set(train_df["PRODUCT_TYPE_ID"].unique())
for i in a:
    if i not in b:
        print(test_df[test_df["PRODUCT_TYPE_ID"] == i])

In [ ]:
dimension_pattern = r'\b(\d+(\.\d+)?)(\s?[xX]\s?)(\d+(\.\d+)?)(\s?[xX]\s?)(\d+(\.\d+)?)(\s?\b(cm|mm|inches|inch)\b)'
dimension_pattern = f'\\b(\\d+(\\.\\d+)?)(\\s*[xX*]\\s*)(\\d+(\\.\\d+)?)(\\s*(?:[xX*]\\s*\\d+(\\.\\d+)?\\s*(?:\\b(cm|mm|in(ch(es)?)?)\\b)?)?)?'
df = test_df[test_df['BULLET_POINTS'].str.contains(dimension_pattern, regex = True)]

In [ ]:
matches = df['BULLET_POINTS'].str.extract(dimension_pattern)

In [ ]:
matches

In [ ]:
df = df.join(matches)
df

In [ ]:
df[8] = df[8].fillna('in')
df.loc[df[8].str.contains('in'), 0] = df.loc[df[8].str.contains('in'), 0].apply(lambda x: float(x) * 100)
df.loc[df[8].str.contains('cm'), 0] = df.loc[df[8].str.contains('cm'), 0].apply(lambda x: float(x) / 2.54 * 100)
df.loc[df[8].str.contains('mm'), 0] = df.loc[df[8].str.contains('mm'), 0].apply(lambda x: float(x) / 25.4 * 100)

In [ ]:
df

In [ ]:
groups = group_by_product_id(train_df)
groups = list(groups.items())
groups.sort(key = lambda x: len(x[1]), reverse = True)
type_to_mean = {}
for key, value in groups:
    type_to_mean[str(key)] = value["PRODUCT_LENGTH"].median()
answer = {}
for i in range(len(test_df)):
    data = test_df.iloc[i]
    pid = data["PRODUCT_ID"]
    key = str(data["PRODUCT_TYPE_ID"])
    value = 1
    if key in type_to_mean.keys():
        value = type_to_mean[key]
    answer[pid] = value

In [ ]:
answer_df = pd.DataFrame(answer.items(), columns = ["PRODUCT_ID", "PRODUCT_LENGTH"])

In [ ]:
# Merge df1 and df2 on the "PRODUCT_ID" column
merged_df = pd.merge(answer_df[["PRODUCT_ID", "PRODUCT_LENGTH"]], df[["PRODUCT_ID", 0]], on="PRODUCT_ID", how="left")

# Replace missing values in "PRODUCT_LENGTH_x" with the corresponding values from "PRODUCT_LENGTH_y"
merged_df[0].fillna(merged_df["PRODUCT_LENGTH"], inplace=True)

# Drop the "PRODUCT_LENGTH_y" column
merged_df.drop(columns=["PRODUCT_LENGTH"], inplace=True)

# Rename the "PRODUCT_LENGTH_x" column to "PRODUCT_LENGTH"
merged_df.rename(columns={0: "PRODUCT_LENGTH"}, inplace=True)

# Update the "PRODUCT_LENGTH" column in df1 with the values from the merged dataframe
answer_df["PRODUCT_LENGTH"] = merged_df["PRODUCT_LENGTH"]

In [ ]:
answer_df

In [ ]:
sample_df = pd.read_csv("sample_submission.csv")
real_answer = {}
factor = 0.95

for i in range(len(answer_df)):
    from_sample = sample_df.iloc[i]
    from_ours = answer_df.iloc[i]
    v1 = from_sample["PRODUCT_LENGTH"]
    v2 = float(from_ours["PRODUCT_LENGTH"])
    pid = int(from_sample["PRODUCT_ID"])
    if v2 >= v1 + factor * v1 or v2 <= v1 - factor * v1:
        real_answer[pid] = v1
    else:
        real_answer[pid] = v2

real_answer_df = pd.DataFrame(real_answer.items(), columns = ["PRODUCT_ID", "PRODUCT_LENGTH"])
real_answer_df.to_csv("submission.csv", index = False)

In [ ]:
real_answer_df